<a href="https://colab.research.google.com/github/LordFunghi/imageGenerator/blob/main/flux.1-schnell_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.safetensors -d /content/TotoroUI/models/unet -o flux1-schnell.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-schnell.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 14652, done.
remote: Total 14652 (delta 0), reused 0 (delta 0), pack-reused 14652 (from 1)
Receiving objects: 100% (14652/14652), 21.12 MiB | 8.01 MiB/s, done.
Resolving deltas: 100% (9820/9820), done.
/content/TotoroUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s e

In [5]:
import random
from PIL import Image
import numpy as np

# Define the animal names you want to iterate over.
animals = ["dog", "cat", "bear", "mouse"]

width = 1024
height = 1024
steps = 4
sampler_name = "euler"
scheduler = "simple"

with torch.inference_mode():
    # Iterate over each ordered pair (animal1, animal2).
    for animal1 in animals:
        for animal2 in animals:
            # Create the prompt using the two animal names.
            positive_prompt = (
                f"A realistic portrait of a {animal1} looking straight ahead at the center. "
                f"Another {animal2}'s portrait partially overlaps the first one at the bottom, "
                f"with its head cut off slightly as if it is intruding into the frame. "
                "The background is simple and neutral to keep the focus on the animals."
            )

            # Generate a random seed for each image.
            seed = random.randint(0, 18446744073709551615)
            print(f"Generating image for {animal1} & {animal2} with seed:", seed)

            # Encode the prompt.
            cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
            cond = [[cond, {"pooled_output": pooled}]]

            # Prepare noise, guider, and sampler.
            noise = RandomNoise.get_noise(seed)[0]
            guider = BasicGuider.get_guider(unet, cond)[0]
            sampler = KSamplerSelect.get_sampler(sampler_name)[0]
            sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
            latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

            # Run the sampling process.
            sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
            model_management.soft_empty_cache()

            # Decode the latent image.
            decoded = VAEDecode.decode(vae, sample)[0].detach()
            image = Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])

            # Save the image with a unique filename.
            image.save(f"/content/flux_{animal1}_{animal2}.png")


Generating image for dog & dog with seed: 3524747801757735981


  0%|          | 0/4 [00:00<?, ?it/s]

Generating image for dog & cat with seed: 7672105054196136684


  0%|          | 0/4 [00:00<?, ?it/s]

Generating image for dog & bear with seed: 1865913260895812510


...

  0%|          | 0/4 [00:00<?, ?it/s]

Generating image for bear & cat with seed: 13130179939089200887


  0%|          | 0/4 [00:00<?, ?it/s]

Generating image for bear & bear with seed: 2556539332527288393


  0%|          | 0/4 [00:00<?, ?it/s]